### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
# ligas = temporada[temporada['Date'] == data_var]['League'].unique().tolist()
# temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos.to_csv(f'jogos_processados/{data_var}.csv', index=False)

jogos[['Date', 'League', 'Time', 'Home', 'Away', 
       'Odds_H', 'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', ]]

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A
5893,2023-11-09,Brasil Nbb,19:00,Cerrado,Caxias do Sul,1.87,1.89,147.5,1.85,1.87,-1.5,1.99,1.73
5894,2023-11-09,Brasil Nbb,19:30,Bauru,Corinthians Paulista,1.66,2.18,151.5,1.89,1.87,-3.5,2.00,1.72
5895,2023-11-09,Brasil Nbb,19:30,Brasília,União Corinthians,1.62,2.26,159.5,1.88,1.86,-4.5,2.08,1.67
5896,2023-11-09,Brasil Nbb,20:00,Pinheiros,Paulistano,2.09,1.71,149.5,1.81,1.85,1.5,1.91,1.80
5897,2023-11-09,Eua Nba,00:00,Denver Nuggets,Golden State Warriors,1.61,2.54,225.5,1.82,2.11,-4.5,2.06,1.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5983,2023-11-09,Rússia Liga Vtb United,13:30,MBA Moscow,Enisey,1.65,2.29,157.5,1.81,1.94,-4.5,2.01,1.77
5984,2023-11-09,Sérvia Primeira Liga,15:00,Subotica,Tamis,1.12,5.53,167.5,1.82,1.84,-13.5,2.05,1.68
5985,2023-11-09,Libano Division 1,11:45,NSA,Al-Riyadi,11.00,1.02,160.5,1.82,1.84,16.5,2.10,1.65
5986,2023-11-09,Mundo Jogos Olímpicos,22:00,Porto Rico F,Colombia F,1.33,3.43,133.5,1.80,1.95,-7.5,2.01,1.75


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
4,00:00,Eua Nba,Denver Nuggets,Golden State Warriors,225.5,1.82,1.0000,Over
5,00:00,Eua Nba,Sacramento Kings,Portland Trail Blazers,220.5,1.86,1.0000,Over
6,21:00,Eua Nba,Indiana Pacers,Milwaukee Bucks,239.5,1.83,0.9961,Over
7,23:30,Eua Nba,Orlando Magic,Atlanta Hawks,229.5,1.83,0.9964,Over
8,14:15,Europa Euroliga,Olympiakos,Baskonia,158.5,1.80,1.0000,Over


### Handicap positivo V4 (LDA)

In [10]:
# Duplicando o dataset
df_hp_v4 = jogos.copy()

df_hp_v4.loc[(df_hp_v4['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_hp_v4.loc[(df_hp_v4['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_hp_v4['Zebra'] = df_hp_v4['Zebra'].astype('category')

# Odds
df_hp_v4.loc[(df_hp_v4['Zebra']) == 'Casa', 'Odds'] = df_hp_v4['HA_Odds_H']
df_hp_v4.loc[(df_hp_v4['Zebra']) == 'Fora', 'Odds'] = df_hp_v4['HA_Odds_A']

df_hp_v4.reset_index(drop=True, inplace=True)
df_hp_v4.replace([np.inf, -np.inf], 0, inplace=True)

if not df_hp_v4.empty:
    mdl_hp_v4 = load_model('ML/handicap_positivo/v4/v4_handicap_positivo', verbose=False)
    prev = predict_model(mdl_hp_v4, df_hp_v4)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Over_Line'] > 155))]

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/handicap_positivo_v4_{data_var}.csv', index=False)

        print('------------ MÉTODO HANDICAP POSITIVO V4 (LDA) ------------')
        print()

        prev.loc[prev['Zebra'] == 'Casa', 'Bet'] = 'HA Casa'
        prev.loc[prev['Zebra'] == 'Fora', 'Bet'] = 'HA Fora'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO HANDICAP POSITIVO (LDA) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
2,19:30,Brasil Nbb,Brasília,União Corinthians,-4.5,1.67,0.9642,HA Fora
9,14:30,Europa Euroliga,Anadolu Efes,Zalgiris Kaunas,-7.5,1.67,0.9993,HA Fora
10,16:05,Europa Euroliga,Maccabi Tel Aviv,Bayern,-7.5,1.67,0.9993,HA Fora
12,16:30,Europa Euroliga,Partizan,Fenerbahce,-6.5,1.69,1.0000,HA Fora
13,16:45,Europa Euroliga,Real Madrid,Virtus Bologna,-11.5,1.65,0.9998,HA Fora
15,08:35,China Cba,Shanxi Zhongyu,Qingdao,-5.5,1.80,0.9940,HA Fora
20,07:00,Coreia Do Sul Kbl,Mobis Phoebus,Seoul Thunders,-10.5,1.68,0.5111,HA Fora
23,14:30,Dinamarca Liga De Basquete,Bears Academy,Holbaek,-15.5,1.73,0.9948,HA Fora
66,14:30,Finlândia I Divisão A,Pyrinto Tampere 2,Helsingin NMKY,-4.5,1.76,0.6868,HA Fora
73,13:00,Lituânia Nkl,Delikatesas Joniskis,BC Olimpas Palanga,8.5,2.07,0.5683,HA Casa


### Handicap positivo V5 (RF)

In [13]:
# Duplicando o dataset
df_hp_v5 = jogos.copy()

df_hp_v5.loc[(df_hp_v5['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_hp_v5.loc[(df_hp_v5['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_hp_v5['Zebra'] = df_hp_v5['Zebra'].astype('category')

# Odds
df_hp_v5.loc[(df_hp_v5['Zebra']) == 'Casa', 'Odds'] = df_hp_v5['HA_Odds_H']
df_hp_v5.loc[(df_hp_v5['Zebra']) == 'Fora', 'Odds'] = df_hp_v5['HA_Odds_A']

df_hp_v5.reset_index(drop=True, inplace=True)
df_hp_v5.replace([np.inf, -np.inf], 0, inplace=True)

if not df_hp_v5.empty:
    mdl_hp_v5 = load_model('ML/handicap_positivo/v5/v5_handicap_positivo', verbose=False)
    prev = predict_model(mdl_hp_v5, df_hp_v5)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Over_Line'] > 155))]

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/handicap_positivo_v5_{data_var}.csv', index=False)

        print('------------ MÉTODO HANDICAP POSITIVO V5 (RF) ------------')
        print()

        prev.loc[prev['Zebra'] == 'Casa', 'Bet'] = 'HA Casa'
        prev.loc[prev['Zebra'] == 'Fora', 'Bet'] = 'HA Fora'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO HANDICAP POSITIVO V5 (RF) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
2,19:30,Brasil Nbb,Brasília,União Corinthians,-4.5,1.67,0.600,HA Fora
4,00:00,Eua Nba,Denver Nuggets,Golden State Warriors,-4.5,1.87,0.520,HA Fora
8,14:15,Europa Euroliga,Olympiakos,Baskonia,-8.5,1.97,0.530,HA Fora
9,14:30,Europa Euroliga,Anadolu Efes,Zalgiris Kaunas,-7.5,1.67,0.770,HA Fora
10,16:05,Europa Euroliga,Maccabi Tel Aviv,Bayern,-7.5,1.67,0.740,HA Fora
12,16:30,Europa Euroliga,Partizan,Fenerbahce,-6.5,1.69,0.680,HA Fora
13,16:45,Europa Euroliga,Real Madrid,Virtus Bologna,-11.5,1.65,0.850,HA Fora
14,19:00,Argentina Liga A,Instituto de Cordoba,Obras,-7.5,1.68,0.530,HA Fora
20,07:00,Coreia Do Sul Kbl,Mobis Phoebus,Seoul Thunders,-10.5,1.68,0.580,HA Fora
21,07:00,Coreia Do Sul Kbl,Suwon KT,KoGas,-8.5,1.72,0.530,HA Fora


### Handicap positivo V6 (LDA)

In [12]:
# Duplicando o dataset
df_hp_v6 = jogos.copy()

df_hp_v6.loc[(df_hp_v6['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_hp_v6.loc[(df_hp_v6['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_hp_v6['Zebra'] = df_hp_v6['Zebra'].astype('category')

# Odds
df_hp_v6.loc[(df_hp_v6['Zebra']) == 'Casa', 'Odds'] = df_hp_v6['HA_Odds_H']
df_hp_v6.loc[(df_hp_v6['Zebra']) == 'Fora', 'Odds'] = df_hp_v6['HA_Odds_A']

df_hp_v6.reset_index(drop=True, inplace=True)
df_hp_v6.replace([np.inf, -np.inf], 0, inplace=True)

if not df_hp_v6.empty:
    mdl_hp_v6 = load_model('ML/handicap_positivo/v5/v5_handicap_positivo', verbose=False)
    prev = predict_model(mdl_hp_v6, df_hp_v6)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/handicap_positivo_v6_{data_var}.csv', index=False)

        print('------------ MÉTODO HANDICAP POSITIVO V6 (LDA) ------------')
        print()

        prev.loc[prev['Zebra'] == 'Casa', 'Bet'] = 'HA Casa'
        prev.loc[prev['Zebra'] == 'Fora', 'Bet'] = 'HA Fora'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO HANDICAP POSITIVO (LDA) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
0,19:00,Brasil Nbb,Cerrado,Caxias do Sul,-1.5,1.73,0.630,HA Fora
1,19:30,Brasil Nbb,Bauru,Corinthians Paulista,-3.5,1.72,0.555,HA Fora
2,19:30,Brasil Nbb,Brasília,União Corinthians,-4.5,1.67,0.600,HA Fora
3,20:00,Brasil Nbb,Pinheiros,Paulistano,1.5,1.91,0.620,HA Casa
4,00:00,Eua Nba,Denver Nuggets,Golden State Warriors,-4.5,1.87,0.520,HA Fora
...,...,...,...,...,...,...,...,...
87,16:45,Reino Unido Bbl,Sheffield Sharks,Surrey Scorchers,-11.5,1.66,0.720,HA Fora
89,13:00,Rússia Liga Vtb United,Unics Kazan,Runa Basket Moscow,-14.5,1.71,0.730,HA Fora
91,15:00,Sérvia Primeira Liga,Subotica,Tamis,-13.5,1.68,0.520,HA Fora
93,22:00,Mundo Jogos Olímpicos,Porto Rico F,Colombia F,-7.5,1.75,0.630,HA Fora
